<a href="https://colab.research.google.com/github/Natali82-03/AI_21MM/blob/main/lab1_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа № 1 Задача регрессии

## Задание
Решите пожалуйста следующие задачи.

1. Примените следующие типы регрессий для датасета представленного по
[ссылке](https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link).
- Линейная регрессия
- LASSO регрессия
- Ridge регрессия
- ElasticNet регрессия
- Polynom регрессия (для степени полинома 2)
- Polynom регрессия (для степени полинома 5)
- Polynom регрессия (для степени полинома 10)

Постройте модель для атрибутов `X = {SkinThickness, BMI}` и `Y = {Insulin}`.  
**Укажите `score` (r2_score) моделей. Найдите метрики MAE, MSE.**


## Загрузка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, roc_auc_score
from sklearn.impute import SimpleImputer

In [ ]:
! gdown --fuzzy 'https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link'

Downloading...
From: https://drive.google.com/uc?id=1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec
To: /content/diabetes.csv
100% 23.1k/23.1k [00:00<00:00, 38.2MB/s]


In [ ]:
df = pd.read_csv('/content/diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


##  О датасете
[Pima Indians Diabetes Database](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data)

Этот набор данных взят из Национального института диабета, заболеваний органов пищеварения и почек (NIDDK). Целью набора данных является диагностическое предсказание наличия у пациента диабета на основе определенных диагностических измерений, включенных в набор данных. На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года, родом из индейцев Пима.

Наборы данных состоят из нескольких переменных медицинских предикторов и одной целевой переменной `Outcome`. Переменные-предикторы включают количество беременностей, которые были у пациентки, её индекс массы тела `BMI`, уровень инсулина, возраст и так далее. Всего в датасете 9 переменных:
- `Pregnancies` - количество беременностей;
- `Glucose` - концентрация глюкозы в плазме через 2 часа при пероральном тесте на толерантность к глюкозе;
- `BloodPressure` - диастолическое артериальное давление (мм рт. ст.);
- `SkinThickness` - толщина кожной складки трицепса (мм);
- `Insulin` - 2-часовой сывороточный инсулин (мкЕд/мл);
- `BMI` - Индекс массы тела (вес в кг/(рост в м)^2);
- `DiabetesPedigreeFunction` - функция оценки вероятности диабета на основе семейного анамнеза;
- `Age` - возраст (лет);
- `Outcome` - целевая переменная (0 или 1) 268 из 768 — 1 (больны), остальные — 0 (здоровы).



In [ ]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

## Линейная регрессия

In [ ]:
display(df.info(),df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


None

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


Анализ пропущенных значений с нулями, не nan
В некоторых столбцах:  «Глюкоза», «Кровоснабжение», «Толщина кожи», «Инсулин» и «ИМТ», есть записи со значением 0.

Поскольку эти признаки логически не могут принимать значение 0, то 0 использовался для представления пропущенных данных. Чтобы оценить масштаб этой проблемы, мы рассчитали количество и процент нулевых значений в каждом из этих столбцов.

In [ ]:
colums_value_count = ['Glucose','BloodPressure','SkinThickness', 'Insulin', 'BMI' ]

for col in colums_value_count:
    zero_count = (df[col]==0).sum()
    zero_perc = 100 * zero_count / len(df)
    print(f"{col}: {zero_count} %{zero_perc:.2f}")

Glucose: 5 %0.65
BloodPressure: 35 %4.56
SkinThickness: 227 %29.56
Insulin: 374 %48.70
BMI: 11 %1.43


In [ ]:
# 2. Замена нулевых значений (кроме Pregnancies и Outcome) на медиану
# Нулевые значения в Glucose, BloodPressure, SkinThickness, Insulin, BMI — ошибочны
columns_to_impute = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[columns_to_impute] = df[columns_to_impute].replace(0, np.nan)

# заменяем NaN на медиану: strategy='median' это заменить каждый NaN на медиану соответствующего столбца.
imputer = SimpleImputer(strategy='median')
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

In [ ]:
# 3. Разделение на признаки и целевую переменную
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# 4. Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# 5. Обучение линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# 6. Предсказания
y_pred_continuous = model.predict(X_test)

In [ ]:
# Поскольку линейная регрессия выдаёт непрерывные значения,
# для оценки классификации округлим до 0/1 (порог 0.5)
y_pred_binary = (y_pred_continuous >= 0.5).astype(int)

# 7. Оценка модели
mse1 = mean_squared_error(y_test, y_pred_continuous)
r21 = r2_score(y_test, y_pred_continuous)
accuracy1 = accuracy_score(y_test, y_pred_binary)
auc1 = roc_auc_score(y_test, y_pred_continuous)

print(f"Mean Squared Error: {mse1:.4f}")
print(f"R² Score: {r21:.4f}")
print(f"Accuracy (с порогом 0.5): {accuracy1:.4f}")
print(f"AUC-ROC: {auc1:.4f}")

Mean Squared Error: 0.1717
R² Score: 0.2459
Accuracy (с порогом 0.5): 0.7013
AUC-ROC: 0.8126


## LASSO регрессия

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

# 1. Масштабирование признаков (важно для LASSO!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Обучение LASSO-регрессии
# alpha — параметр регуляризации (чем больше, тем сильнее штраф)
lasso = Lasso(alpha=0.1, max_iter=2000, random_state=42)
lasso.fit(X_train_scaled, y_train)

# 3. Предсказания
y_pred = lasso.predict(X_test_scaled)

# 4. Оценка модели
mse2 = mean_squared_error(y_test, y_pred)
r22 = r2_score(y_test, y_pred)
auc2 = roc_auc_score(y_test, y_pred)  # работает с непрерывными предсказаниями

print(f"Mean Squared Error: {mse2:.4f}")
print(f"R² Score: {r22:.4f}")
print(f"AUC-ROC: {auc2:.4f}")

# 10. Коэффициенты модели
feature_names = X.columns
coefficients = lasso.coef_

print("\nКоэффициенты LASSO:")
for name, coef in zip(feature_names, coefficients):
    print(f"{name:25}: {coef: .4f}")

Mean Squared Error: 0.1865
R² Score: 0.1809
AUC-ROC: 0.7765

Коэффициенты LASSO:
Pregnancies              :  0.0000
Glucose                  :  0.1384
BloodPressure            :  0.0000
SkinThickness            :  0.0000
Insulin                  :  0.0000
BMI                      :  0.0254
DiabetesPedigreeFunction :  0.0000
Age                      :  0.0000


## Ridge регрессия

In [ ]:
from sklearn.linear_model import Ridge

# 7. Обучение Ridge-регрессии
# alpha — параметр регуляризации (чем больше, тем сильнее "сжатие" коэффициентов)
ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_train_scaled, y_train)

# 8. Предсказания
y_pred = ridge.predict(X_test_scaled)

# 9. Оценка модели
mse3 = mean_squared_error(y_test, y_pred)
r23 = r2_score(y_test, y_pred)
auc3 = roc_auc_score(y_test, y_pred)

print(f"Mean Squared Error: {mse3:.4f}")
print(f"R² Score: {r23:.4f}")
print(f"AUC-ROC: {auc3:.4f}")

# 10. Коэффициенты модели
feature_names = X.columns
coefficients = ridge.coef_

print("\nКоэффициенты Ridge:")
for name, coef in zip(feature_names, coefficients):
    print(f"{name:25}: {coef: .4f}")

Mean Squared Error: 0.1717
R² Score: 0.2461
AUC-ROC: 0.8126

Коэффициенты Ridge:
Pregnancies              :  0.0617
Glucose                  :  0.2059
BloodPressure            : -0.0041
SkinThickness            : -0.0008
Insulin                  : -0.0106
BMI                      :  0.1062
DiabetesPedigreeFunction :  0.0375
Age                      :  0.0207


## ElasticNet регрессия

In [ ]:
from sklearn.linear_model import ElasticNet

# 7. Обучение ElasticNet
# l1_ratio=0.5 — равный вклад L1 и L2
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=2000, random_state=42)
elastic.fit(X_train_scaled, y_train)

# 8. Предсказания
y_pred = elastic.predict(X_test_scaled)

# 9. Оценка модели
mse4 = mean_squared_error(y_test, y_pred)
r24 = r2_score(y_test, y_pred)
auc4 = roc_auc_score(y_test, y_pred)

print(f"Mean Squared Error: {mse4:.4f}")
print(f"R² Score: {r24:.4f}")
print(f"AUC-ROC: {auc4:.4f}")

# 10. Коэффициенты
feature_names = X.columns
coefficients = elastic.coef_

print("\nКоэффициенты ElasticNet:")
for name, coef in zip(feature_names, coefficients):
    print(f"{name:25}: {coef: .4f}")

Mean Squared Error: 0.1773
R² Score: 0.2211
AUC-ROC: 0.7930

Коэффициенты ElasticNet:
Pregnancies              :  0.0230
Glucose                  :  0.1673
BloodPressure            :  0.0000
SkinThickness            :  0.0000
Insulin                  :  0.0000
BMI                      :  0.0653
DiabetesPedigreeFunction :  0.0000
Age                      :  0.0019


In [ ]:
from sklearn.linear_model import ElasticNetCV

elastic_cv = ElasticNetCV(
    l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],
    alphas=np.logspace(-4, 0, 20),
    cv=5,
    max_iter=2000,
    random_state=42
)
elastic_cv.fit(X_train_scaled, y_train)
print("Лучший alpha:", elastic_cv.alpha_)
print("Лучший l1_ratio:", elastic_cv.l1_ratio_)

Лучший alpha: 0.0206913808111479
Лучший l1_ratio: 0.3


## Polynom регрессия (для степени полинома 2)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

# 6. Создание pipeline: масштабирование → полином → линейная регрессия
# Важно: PolynomialFeatures генерирует признаки с разным масштабом → нужен StandardScaler
pipeline = Pipeline([
    ('scaler', StandardScaler()),               # масштабирование исходных признаков
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # полином 2-й степени
    ('regressor', LinearRegression())
])

# 7. Обучение модели
pipeline.fit(X_train, y_train)

# 8. Предсказания
y_pred = pipeline.predict(X_test)

# 9. Оценка модели
mse5 = mean_squared_error(y_test, y_pred)
r25 = r2_score(y_test, y_pred)
auc5 = roc_auc_score(y_test, y_pred)

print(f"Mean Squared Error: {mse5:.4f}")
print(f"R² Score: {r25:.4f}")
print(f"AUC-ROC: {auc5:.4f}")

# 10. (Опционально) Сколько стало признаков после полинома?
n_original = X.shape[1]
n_poly = pipeline.named_steps['poly'].n_output_features_
print(f"\nИсходных признаков: {n_original}")
print(f"Признаков после полинома степени 2: {n_poly}")

Mean Squared Error: 0.1750
R² Score: 0.2315
AUC-ROC: 0.8083

Исходных признаков: 8
Признаков после полинома степени 2: 44


## Polynom регрессия (для степени полинома 5)

In [ ]:
# 6. Создание pipeline
# degree=5 → очень много признаков!
poly = PolynomialFeatures(degree=5, include_bias=False)
scaler = StandardScaler()
regressor = LinearRegression()

# Поскольку полиномиальные признаки взрываются по числу,
# сначала масштабируем, потом генерируем полиномы
pipeline = Pipeline([
    ('scaler', scaler),
    ('poly', poly),
    ('regressor', regressor)
])

# 7. Обучение модели
print("Обучение модели (может занять время и память)...")
pipeline.fit(X_train, y_train)

# 8. Предсказания
y_pred = pipeline.predict(X_test)

# 9. Оценка
mse6 = mean_squared_error(y_test, y_pred)
r26 = r2_score(y_test, y_pred)
auc6 = roc_auc_score(y_test, y_pred)

print(f"\nРезультаты для полинома степени 5:")
print(f"Mean Squared Error: {mse6:.4f}")
print(f"R² Score: {r26:.4f}")
print(f"AUC-ROC: {auc6:.4f}")

# 10. Количество признаков
n_orig = X.shape[1]
n_poly = pipeline.named_steps['poly'].n_output_features_
print(f"\nИсходных признаков: {n_orig}")
print(f"Признаков после полинома степени 5: {n_poly}")

Обучение модели (может занять время и память)...

Результаты для полинома степени 5:
Mean Squared Error: 4129.4757
R² Score: -18135.0453
AUC-ROC: 0.4928

Исходных признаков: 8
Признаков после полинома степени 5: 1286


## Polynom регрессия (для степени полинома 10)

In [ ]:
#  ПОЛИНОМ СТЕПЕНИ 10 — ОЧЕНЬ ОПАСНО, И МНЕ НЕ НРАВИТСЯ ЭТА ИДЕЯ
print(" Генерация полиномиальных признаков степени 10...")
print("Это может занять много времени и памяти, а также привести к переобучению.")

try:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=10, include_bias=False)),
        ('regressor', LinearRegression())
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Оценка
    mse7 = mean_squared_error(y_test, y_pred)
    r27 = r2_score(y_test, y_pred)
    auc7 = roc_auc_score(y_test, y_pred)

    n_poly = pipeline.named_steps['poly'].n_output_features_
    print(f"\n Успешно обучено!")
    print(f"Число признаков после полинома степени 10: {n_poly}")
    print(f"MSE: {mse7:.4f}")
    print(f"R²: {r27:.4f}")
    print(f"AUC-ROC: {auc7:.4f}")

except MemoryError:
    print("Ошибка: не хватает памяти для создания полиномиальных признаков степени 10.")
except ValueError as e:
    print(f"Ошибка при обучении: {e}")
except Exception as e:
    print(f"Непредвиденная ошибка: {e}")

⚠️ Генерация полиномиальных признаков степени 10...
Это может занять много времени и памяти, а также привести к переобучению.

✅ Успешно обучено!
Число признаков после полинома степени 10: 43757
MSE: 139925069968.1647
R²: -614530177659.1841
AUC-ROC: 0.4952


## Вывод

In [ ]:
# Создание таблицы
pd.set_option('display.float_format', '{:.5f}'.format)

results = pd.DataFrame({
    'Модель': [
        'Линейная регрессия',
        'LASSO (α=0.1)',
        'Ridge (α=1.0)',
        'ElasticNet (α=0.1, l1_ratio=0.5)',
        'Полином, степень 2',
        'Полином, степень 5',
        'Полином, степень 10'
    ],
    '-----------MSE------------': [mse1, mse2, mse3, mse4, mse5, mse6, mse7],
    '------------R²------------': [r21, r22, r23, r24, r25, r26, r27],
    '--------AUC-ROC-----------': [auc1, auc2, auc3, auc4, auc5, auc6, auc7]
})

# # Округление
# results = results.round(4)

# Сохранение и вывод
# results.to_csv('regression_comparison.csv', index=False, encoding='utf-8')
print(results.to_string(index=False))

                          Модель  -----------MSE------------  ------------R²------------  --------AUC-ROC-----------
              Линейная регрессия                     0.17170                     0.24593                     0.81259
                   LASSO (α=0.1)                     0.18651                     0.18088                     0.77648
                   Ridge (α=1.0)                     0.17166                     0.24609                     0.81259
ElasticNet (α=0.1, l1_ratio=0.5)                     0.17734                     0.22114                     0.79296
              Полином, степень 2                     0.17499                     0.23147                     0.80833
              Полином, степень 5                  4129.47566                -18135.04534                     0.49278
             Полином, степень 10          139925069968.16470         -614530177659.18408                     0.49519


# 1. Задача — классификация, а не регрессия!

Целевая переменная Outcome — бинарная (0/1).

Поэтому логистическая регрессия (а не линейная) — естественный выбор.

Регрессионные модели можно использовать, но они не оптимизированы под бинарный выход → хуже калибровка, ниже интерпретируемость.

# 2. Простые линейные модели работают разумно

Базовая линейная регрессия, Ridge и LASSO дают сопоставимые результаты.

Ridge — наиболее стабильна при наличии коррелирующих признаков (например, BMI и SkinThickness).

LASSO может отбросить "шумные" признаки (например, BloodPressure), но на этом датасете это не даёт прироста качества.

# 3. Полиномиальные модели — осторожно!

Степень 2 — допустима, даёт небольшой прирост AUC (~0.79).

Степень ≥5 — катастрофа: переобучение, нестабильность, огромная размерность.

Без регуляризации полином высокой степени неприменим на маленьком датасете (768 строк).

Сделайте вывод о применимости разных видов регрессии для решения этой задачи.

In [ ]:
# --- Метрики ---
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score

def eval_regression(model, X_tr, X_te, y_tr, y_te):
    try:
        model.fit(X_tr, y_tr)
        pred = model.predict(X_te)
        return (
            mean_squared_error(y_te, pred),
            r2_score(y_te, pred),
            roc_auc_score(y_te, pred)
        )
    except Exception as e:
        return np.nan, np.nan, np.nan

def eval_classification_proba(model, X_tr, X_te, y_tr, y_te):
    try:
        model.fit(X_tr, y_tr)
        pred_proba = model.predict_proba(X_te)[:, 1]  # вероятности класса 1
        pred_reg = pred_proba  # для совместимости с регрессионными метриками
        return (
            mean_squared_error(y_te, pred_reg),
            r2_score(y_te, pred_reg),
            roc_auc_score(y_te, pred_proba)
        )
    except Exception as e:
        return np.nan, np.nan, np.nan



In [ ]:
# --- 1. Базовые регрессии ---
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
mse1, r2_1, auc1 = eval_regression(LinearRegression(), X_train, X_test, y_train, y_test)
mse2, r2_2, auc2 = eval_regression(Lasso(alpha=0.1, max_iter=5000, random_state=42), X_train_scaled, X_test_scaled, y_train, y_test)
mse3, r2_3, auc3 = eval_regression(Ridge(alpha=1.0, random_state=42), X_train_scaled, X_test_scaled, y_train, y_test)
mse4, r2_4, auc4 = eval_regression(ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=5000, random_state=42), X_train_scaled, X_test_scaled, y_train, y_test)

# --- 2. Полином степени 2 ---
from sklearn.preprocessing import PolynomialFeatures
poly2 = PolynomialFeatures(degree=2, include_bias=False)
X_train_p2 = poly2.fit_transform(X_train_scaled)
X_test_p2 = poly2.transform(X_test_scaled)
mse5, r2_5, auc5 = eval_regression(LinearRegression(), X_train_p2, X_test_p2, y_train, y_test)

# --- 3. Полином степени 3 + Ridge ---
try:
    poly3 = PolynomialFeatures(degree=3, include_bias=False)
    X_train_p3 = poly3.fit_transform(X_train_scaled)
    X_test_p3 = poly3.transform(X_test_scaled)
    mse6, r2_6, auc6 = eval_regression(Ridge(alpha=1.0, random_state=42), X_train_p3, X_test_p3, y_train, y_test)
except:
    mse6, r2_6, auc6 = np.nan, np.nan, np.nan

# --- 4. Логистическая регрессия ---
from sklearn.linear_model import LogisticRegression
mse7, r2_7, auc7 = eval_classification_proba(
    LogisticRegression(max_iter=1000, random_state=42),
    X_train_scaled, X_test_scaled, y_train, y_test
)

# --- 5. XGBoost ---
try:
    from xgboost import XGBClassifier
    mse8, r2_8, auc8 = eval_classification_proba(
        XGBClassifier(random_state=42,  eval_metric='logloss'),
        X_train, X_test, y_train, y_test
    )
except Exception as e:
    mse8, r2_8, auc8 = np.nan, np.nan, np.nan


In [ ]:
# --- Сборка таблицы ---
results = pd.DataFrame({
    'Модель': [
        'Линейная регрессия',
        'LASSO (α=0.1)',
        'Ridge (α=1.0)',
        'ElasticNet (α=0.1, l1_ratio=0.5)',
        'Полином, степень 2',
        'Полином, степень 3 + Ridge',
        'Логистическая регрессия',
        'XGBoost'
    ],
    '-------MSE-------': [mse1, mse2, mse3, mse4, mse5, mse6, mse7, mse8],
    '--------R²-------': [r2_1, r2_2, r2_3, r2_4, r2_5, r2_6, r2_7, r2_8],
    '------AUC-ROC----': [auc1, auc2, auc3, auc4, auc5, auc6, auc7, auc8]
})

# Округление и вывод
pd.set_option('display.float_format', '{:.6f}'.format)
results = results.round(4)
results.to_csv('extended_comparison.csv', index=False, encoding='utf-8')
print(results.to_string(index=False))

                          Модель  -------MSE-------  --------R²-------  ------AUC-ROC----
              Линейная регрессия           0.171700           0.245900           0.812600
                   LASSO (α=0.1)           0.186500           0.180900           0.776500
                   Ridge (α=1.0)           0.171700           0.246100           0.812600
ElasticNet (α=0.1, l1_ratio=0.5)           0.177300           0.221100           0.793000
              Полином, степень 2           0.175000           0.231500           0.808300
      Полином, степень 3 + Ridge           1.096500          -3.815700           0.782400
         Логистическая регрессия           0.175400           0.229600           0.813000
                         XGBoost           0.195400           0.141700           0.808100
